In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests11 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:
def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000
adapt_c = False 
data_tuned_epsilon = False # note, this changes both sw test and sw bootstrap...
epsilon = .5

size weirdness rn...

In [5]:
a = 0.25
num_params=9

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 0.96 & 0.84 & 0.96 & 0.95   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.09 & 0.02 & 0.02   \\
Model 2 & 0.06 & 0.06 & 0.03 & 0.07 & 0.01 & 0.03   \\
\hline
\end{tabular}
(array([0.936, 0.006, 0.058]), array([0.936, 0.006, 0.058]), array([0.962, 0.01 , 0.028]), array([0.844, 0.088, 0.068]), array([0.964, 0.024, 0.012]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.954, 0.02 , 0.026]), -4.253783240407408, 10.805160818792672, 11.057502040902992)


# power 0.... need to see...

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [7]:
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.83 & 0.83 & 0.69 & 0.50 & 0.50 & 0.54   \\
Model 1 & 0.17 & 0.17 & 0.31 & 0.50 & 0.50 & 0.46   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.83, 0.17, 0.  ]), array([0.834, 0.166, 0.   ]), array([0.688, 0.312, 0.   ]), array([0.498, 0.5  , 0.002]), array([0.502, 0.498, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.54, 0.46, 0.  ]), 6.539484846457685, 5.283515152843739, 5.34261749239963)


In [8]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.80 & 0.81 & 0.71 & 0.52 & 0.51 & 0.51   \\
Model 1 & 0.20 & 0.19 & 0.28 & 0.48 & 0.49 & 0.49   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.8, 0.2, 0. ]), array([0.806, 0.194, 0.   ]), array([0.714, 0.284, 0.002]), array([0.516, 0.476, 0.008]), array([0.506, 0.494, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.514, 0.486, 0.   ]), 6.852713267378343, 5.055891651284302, 5.3414121033999695)


In [9]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.77 & 0.78 & 0.67 & 0.50 & 0.48 & 0.51   \\
Model 1 & 0.23 & 0.22 & 0.33 & 0.50 & 0.52 & 0.49   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.772, 0.228, 0.   ]), array([0.78, 0.22, 0.  ]), array([0.674, 0.326, 0.   ]), array([0.498, 0.5  , 0.002]), array([0.484, 0.516, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.512, 0.488, 0.   ]), 7.263679064644158, 5.282837615550324, 5.386694929672163)


In [10]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.82 & 0.83 & 0.72 & 0.51 & 0.51 & 0.52   \\
Model 1 & 0.18 & 0.17 & 0.28 & 0.48 & 0.48 & 0.48   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.82 , 0.178, 0.002]), array([0.828, 0.17 , 0.002]), array([0.718, 0.278, 0.004]), array([0.506, 0.482, 0.012]), array([0.514, 0.484, 0.002]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.522, 0.478, 0.   ]), 6.785245699087152, 5.069532093520673, 5.401912653795594)


In [11]:
data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/50,nobs**(1/10) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.80 & 0.81 & 0.71 & 0.51 & 0.51 & 0.53   \\
Model 1 & 0.20 & 0.19 & 0.29 & 0.49 & 0.49 & 0.47   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.8, 0.2, 0. ]), array([0.806, 0.194, 0.   ]), array([0.71, 0.29, 0.  ]), array([0.506, 0.492, 0.002]), array([0.508, 0.492, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.53, 0.47, 0.  ]), 6.939815979897297, 4.926428853523473, 5.390198932510674)


# a  = .25, k= 9

In [12]:
a = 0.25
num_params=9



In [13]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.94 & 0.96 & 0.82 & 0.93 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.02 & 0.13 & 0.06 & 0.02   \\
Model 2 & 0.07 & 0.06 & 0.03 & 0.04 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.932, 0.   , 0.068]), array([0.936, 0.   , 0.064]), array([0.956, 0.016, 0.028]), array([0.824, 0.134, 0.042]), array([0.932, 0.056, 0.012]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.97 , 0.018, 0.012]), -4.047027168632682, 5.594126116496583, 5.969769844181517)


In [14]:
nobs = 500

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.97 & 0.82 & 0.95 & 0.96   \\
Model 1 & 0.01 & 0.01 & 0.03 & 0.12 & 0.04 & 0.03   \\
Model 2 & 0.06 & 0.06 & 0.01 & 0.06 & 0.00 & 0.01   \\
\hline
\end{tabular}
(array([0.926, 0.014, 0.06 ]), array([0.926, 0.014, 0.06 ]), array([0.968, 0.026, 0.006]), array([0.824, 0.12 , 0.056]), array([0.954, 0.042, 0.004]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.962, 0.026, 0.012]), -4.008072244118264, 8.190487092817571, 8.017515696248513)


In [15]:
nobs = 100

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.96 & 0.98 & 0.84 & 0.91 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.14 & 0.08 & 0.01   \\
Model 2 & 0.10 & 0.04 & 0.01 & 0.02 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.9  , 0.004, 0.096]), array([0.962, 0.   , 0.038]), array([0.978, 0.012, 0.01 ]), array([0.838, 0.142, 0.02 ]), array([0.908, 0.084, 0.008]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.99 , 0.008, 0.002]), -3.935354878815645, 4.060837934507989, 4.362577099894972)


# a  = .25, k= 4, something not right?

In [16]:
a = 0.25
num_params=4



In [17]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.95 & 0.96 & 0.81 & 0.95 & 0.98   \\
Model 1 & 0.02 & 0.02 & 0.02 & 0.11 & 0.04 & 0.01   \\
Model 2 & 0.03 & 0.03 & 0.02 & 0.08 & 0.02 & 0.00   \\
\hline
\end{tabular}
(array([0.948, 0.018, 0.034]), array([0.948, 0.018, 0.034]), array([0.964, 0.016, 0.02 ]), array([0.806, 0.112, 0.082]), array([0.948, 0.036, 0.016]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.982, 0.014, 0.004]), -1.0702780388122912, 5.714992461515259, 5.65314835443659)


# a  = .25, K = 19

In [18]:
a = 0.25
num_params=19


In [19]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.73 & 0.82 & 0.95 & 0.83 & 0.90 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.15 & 0.09 & 0.00   \\
Model 2 & 0.27 & 0.18 & 0.04 & 0.02 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.726, 0.   , 0.274]), array([0.824, 0.   , 0.176]), array([0.954, 0.01 , 0.036]), array([0.828, 0.148, 0.024]), array([0.9  , 0.092, 0.008]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.984, 0.004, 0.012]), -9.633390327927245, 6.202578961400796, 6.657394791287272)


# evidence of power

In [20]:
nobs = 500

a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 0.85 & 0.50 & 0.55 & 0.64   \\
Model 1 & 0.02 & 0.02 & 0.15 & 0.49 & 0.45 & 0.36   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.98, 0.02, 0.  ]), array([0.98, 0.02, 0.  ]), array([0.846, 0.154, 0.   ]), array([0.5 , 0.49, 0.01]), array([0.552, 0.448, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.644, 0.356, 0.   ]), 1.7776924498061346, 5.570466341655601, 6.306132153974583)


In [21]:
nobs = 100

a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.95 & 1.00 & 0.79 & 0.36 & 0.35 & 0.72   \\
Model 1 & 0.02 & 0.00 & 0.21 & 0.64 & 0.65 & 0.28   \\
Model 2 & 0.03 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.946, 0.022, 0.032]), array([1., 0., 0.]), array([0.794, 0.206, 0.   ]), array([0.36, 0.64, 0.  ]), array([0.35, 0.65, 0.  ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.716, 0.284, 0.   ]), 0.21428519709101707, 5.7411596209322715, 5.872976345230389)


In [22]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.59 & 0.59 & 0.66 & 0.53 & 0.58 & 0.55   \\
Model 1 & 0.41 & 0.41 & 0.34 & 0.46 & 0.42 & 0.45   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.588, 0.412, 0.   ]), array([0.588, 0.412, 0.   ]), array([0.66, 0.34, 0.  ]), array([0.53, 0.46, 0.01]), array([0.576, 0.424, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.546, 0.454, 0.   ]), 9.055470820896911, 4.821261586186798, 4.851553139727758)


In [23]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.99 & 0.85 & 0.47 & 0.45 & 0.68   \\
Model 1 & 0.02 & 0.01 & 0.15 & 0.52 & 0.55 & 0.32   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.974, 0.022, 0.004]), array([0.986, 0.014, 0.   ]), array([0.846, 0.152, 0.002]), array([0.474, 0.524, 0.002]), array([0.452, 0.548, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.684, 0.316, 0.   ]), 1.0407405008094825, 5.64225435484289, 6.1470258445922665)


# evidence of power 2

In [24]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.59 & 0.61 & 0.79 & 0.91 & 0.65   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.04 & 0.01 & 0.00   \\
Model 2 & 0.99 & 0.41 & 0.39 & 0.17 & 0.08 & 0.35   \\
\hline
\end{tabular}
(array([0.008, 0.   , 0.992]), array([0.59, 0.  , 0.41]), array([0.614, 0.   , 0.386]), array([0.786, 0.042, 0.172]), array([0.91 , 0.006, 0.084]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.652, 0.   , 0.348]), -20.191497678299196, 5.57739198142824, 6.151217098747267)


In [25]:
nobs = 500


a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.41 & 0.73 & 0.77 & 0.93 & 0.69   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.05 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.59 & 0.27 & 0.18 & 0.07 & 0.31   \\
\hline
\end{tabular}
(array([0.01, 0.  , 0.99]), array([0.408, 0.   , 0.592]), array([0.73, 0.  , 0.27]), array([0.772, 0.046, 0.182]), array([0.932, 0.002, 0.066]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.69, 0.  , 0.31]), -19.84235910744811, 5.762066526440851, 6.217299436371496)


In [26]:
nobs = 100


a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.92 & 0.37 & 0.78 & 0.83 & 0.52   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
Model 2 & 1.00 & 0.08 & 0.63 & 0.21 & 0.17 & 0.48   \\
\hline
\end{tabular}
(array([0.004, 0.   , 0.996]), array([0.916, 0.   , 0.084]), array([0.37, 0.  , 0.63]), array([0.776, 0.014, 0.21 ]), array([0.83 , 0.002, 0.168]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.524, 0.   , 0.476]), -21.303344965049543, 5.411083786285382, 5.819936860238864)


In [27]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.22 & 0.22 & 0.60 & 0.56 & 0.66 & 0.58   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.00 & 0.00   \\
Model 2 & 0.78 & 0.78 & 0.40 & 0.42 & 0.34 & 0.42   \\
\hline
\end{tabular}
(array([0.218, 0.   , 0.782]), array([0.224, 0.   , 0.776]), array([0.604, 0.   , 0.396]), array([0.562, 0.02 , 0.418]), array([0.658, 0.   , 0.342]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.578, 0.   , 0.422]), -12.11843826523959, 4.726585112229164, 4.906366468917083)


In [28]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.59 & 0.64 & 0.81 & 0.92 & 0.69   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.41 & 0.36 & 0.17 & 0.08 & 0.31   \\
\hline
\end{tabular}
(array([0.006, 0.   , 0.994]), array([0.588, 0.   , 0.412]), array([0.64, 0.  , 0.36]), array([0.81 , 0.018, 0.172]), array([0.92 , 0.002, 0.078]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.69, 0.  , 0.31]), -19.84004637559656, 5.318072674946279, 6.110502535554391)
